### RAG with Tabalar Data and Vector Memory


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName('Chatbot_rag_v2') \
    .config("spark.jars", "/opt/spark/jars/iceberg-spark-runtime-3.5_2.12-1.6.0.jar") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hive") \
    .config("spark.sql.catalog.local.warehouse", "s3a://datalake/iceberg") \
    .getOrCreate()

#Ajuste de log WARN log para ERROR
spark.sparkContext.setLogLevel("ERROR")

In [ ]:
# !pip install langchain==0.2.17
# !pip install langchain_community==0.2.19
# !pip install -qU langchain-ollama==0.1.3
# !pip install -qU langchain-qdrant==0.1.4

In [ ]:
import os
from dotenv import load_dotenv


### Visualizar e pegar uma amostra dos dados

In [ ]:
spark.sql("Select * from iceberg.silver.tbl_silver_olhovivo").limit(10).show()

In [ ]:
df = spark.sql("""
    Select
    c,
    cl,
    sl,
    lt0,
    lt1,
    qv
    
    from iceberg.silver.tbl_silver_olhovivo """
).limit(10)

df.createOrReplaceTempView("vw_silver_olhovivo")

spark.sql("select * from vw_silver_olhovivo").show()

## Funções Auxiliares

In [ ]:
import re

def clear_sql(genereted_sql):
    """Remove markdown code blocks"""
    sql = re.sub(r"```sql|```", "", genereted_sql, flags=re.IGNORECASE).strip()
    return sql

In [ ]:
def get_metadata(table_name):
    df = spark.sql(f"SELECT * FROM {table_name} LIMIT 1;")
    columns = "\n".join([f"- {f.name}: {f.dataType.simpleString()}" for f in df.schema])
    return f"Tabela: {table_name}\n\nColunas:\n{columns}"


## Qdrant Memory

In [ ]:
load_dotenv('../.env')
OLLAMA_API_URL = os.getenv("OLLAMA_API_URL")


In [ ]:
from langchain_ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(model="mistral:latest", base_url=OLLAMA_API_URL)

In [ ]:
# Cria coleção para armazenar os embeddings 

from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.models import Distance, VectorParams, PointStruct, Filter, FieldCondition, MatchValue
from langchain_core.documents import Document
import uuid

client = QdrantClient(":memory:")

collection_name ="olho_vivo"

client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=4096, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embedding,
)

retriever = vector_store.as_retriever(search_kwargs={"k": 2})

In [ ]:
%run ./Memory.ipynb

In [ ]:
qdrant_memory = QdrantMemory(client, embedding)

## Iniciar Mistral 7B

In [ ]:
from langchain_ollama import ChatOllama

# llm = ChatOllama(
#     model="mistral:latest", 
#     base_url=OLLAMA_API_URL,
#     temperature = 0.3,
 
# )

llm = ChatOllama(
    model="mistral:latest", 
    base_url=OLLAMA_API_URL,
    temperature=0.3,
    num_predict=200,
    top_k=20,
    repeat_penalty=1.2
    
) 

### Configurar Promps: Roles System e Human

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Prompt para gerar SQL (Spark)
prompt_sql = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("Você é um especialista em dados. Gere apenas a consulta SQL."),
    HumanMessagePromptTemplate.from_template(
        "Com base na estrutura da tabela abaixo:\n\n{schema}\n\n"
        "Escreva uma consulta SQL (somente a SQL) para responder:\n{question}"
    )
])


# Prompt para retornar resultado ao usuario
prompt_response = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("Você é um assistente de dados."),
    HumanMessagePromptTemplate.from_template(
        "Pergunta: {question}\n\nResultado da consulta:\n{result}\n\n"
        "Gere uma resposta clara e amigável para o usuário contendo apenas os resultados da consulta."
    )
])


In [ ]:
# Função para gerar resposta com RAG (Tabela + Qdrant)
def augmented_response(question, table_name):
    print(f"\n💬 question : {question}")

    #Obter metadados da tabela
    schema_txt = get_metadata(table_name)

    # Buscar embeddings no Qdrant (Memoria)
    docs = retriever.invoke(question)
    context = "\n".join([doc.page_content for doc in docs])

    # Gerar o SQL da query com base na pergunta
    sql_chain = prompt_sql | llm
    sql_result = sql_chain.invoke({
        "question": question,
        "schema": schema_txt,
        "context": context
    }).content.strip()


    sql_query=clear_sql(sql_result)
    print(f"\n🤖💡 Generated SQL: {sql_query}")

    # Executar query no Spark
    try:

        result_df = spark.sql(sql_query).toPandas().to_dict(orient="records")
    except Exception as e:
        print(f"\n❌ Erro na execução da SQL: {e}")
        return

    # Gera resposta amigável para retornar ao usuario
    response_chain = prompt_response | llm
    response = response_chain.invoke({
        "question": question,
        "result": result_df
    }).content.strip()

    print(f"\n🤖 response : {response}")


    # Armazena pergunta + SQL + resultado + score na memoria Qdrant
    qdrant_memory.instruct(
        question, 
        sql_query, 
        metadata={
            "table": table_name, 
            "result": result_df, 
            "schema": schema_txt, 
            "score": 1}
    )

    return response

In [ ]:
resp = augmented_response(question, table)

### Listar Exemplos de Consultas

In [ ]:
qdrant_memory.list_embedding_content()

In [ ]:
qdrant_memory.list_scored_point("question here")

## "Ensinar" refinar comportamento manualmente

In [ ]:
sql_query =""" """

In [ ]:
spark.sql(sql_query).show()

In [ ]:
table_name = "table_name"
pergunta = ""
schema_txt = get_metadata(table_name)

qdrant_memory.instruct(
    pergunta,
    sql_query, 
    metadados=metadata={"table": table_name, "result": result_df, "schema": schema_txt, "score": 1}
)